In [ ]:
import glob
import numpy as np
from PIL import Image
import torch
from transformers import AutoProcessor, AutoModelForVision2Seq

/Users/jeremiahetiosaomeike/research_projects/vla/LIBERO/libero_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/jeremiahetiosaomeike/research_projects/vla/LIBERO/libero_venv/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x10769f700>>
Traceback (most recent call last):
  File "/Users/jeremiahetiosaomeike/research_projects/vla/LIBERO/libero_venv/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
Keyboa

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device_map = 'cuda' if torch.cuda.is_available() else 'cpu'
open_vla_path = ... # Open VLA Path Here

processor = AutoProcessor.from_pretrained(open_vla_path, trust_remote_code=True, device_map=device_map)
vla = AutoModelForVision2Seq.from_pretrained( 
   open_vla_path,
   # attn_implementation="flash_attention_2",  # [Optional] Requires `flash_attn`
   torch_dtype=torch.bfloat16,
   low_cpu_mem_usage=True,
   trust_remote_code=True
)
vla = vla.to(device)  

In [ ]:
vla_data_path_npys = ... # '/scratch/gpfs/jomeike/vla_run_data/*.npy' Dataset Path Here
npy_files = glob.glob(vla_data_path_npys)

for npy_file in npy_files:
    dat = np.load(npy_file, allow_pickle=True)
    for dct in dat:
        image_image = Image.fromarray(dct['image'])
        lang_instruction = dct['language_instruction']
        inputs = processor(lang_instruction, image_image).to(device, dtype=torch.bfloat16)
       
        with torch.amp.autocast('cuda'):
            with torch.no_grad():
                gends = vla.generate(**inputs,
                                max_new_tokens=vla.get_action_dim('bridge_orig'),
                                return_dict_in_generate=True,
                                output_hidden_states=True)
                
                # pre-generation hidden states 
                hidden_states_before_generation = gends['hidden_states'][0]
                hidden_state_last_layer = hidden_states_before_generation[-1]
                hidden_state_at_last_token = hidden_state_last_layer[:, -1, :]
                dct['hidden_state_before_gen'] = hidden_state_at_last_token.cpu().float().numpy()
                
                # post-generation hidden states
                hidden_state_post_generation = gends['hidden_states'][-1]
                hidden_state_post_generation = hidden_state_post_generation[-1][0]
                dct['hidden_state_post_gen'] = hidden_state_post_generation.cpu().float().numpy()
        
    np.save(npy_file, dat)
    print(f'Saved npy file {npy_file}')

In [ ]:
vla_data_path_validation_npys = ... # '/scratch/gpfs/jomeike/vla_run_data/*.npy' Dataset Path Here
npy_validation_files = glob.glob(vla_data_path_validation_npys)

for npy_file in npy_files:
    dat = np.load(npy_file, allow_pickle=True)
    for dct in dat:
        image_image = Image.fromarray(dct['image'])
        lang_instruction = dct['language_instruction']
        inputs = processor(lang_instruction, image_image).to(device, dtype=torch.bfloat16)
       
        with torch.amp.autocast('cuda'):
            with torch.no_grad():
                gends = vla.generate(**inputs,
                                max_new_tokens=vla.get_action_dim('bridge_orig'),
                                return_dict_in_generate=True,
                                output_hidden_states=True)
                
                # pre-generation hidden states 
                hidden_states_before_generation = gends['hidden_states'][0]
                hidden_state_last_layer = hidden_states_before_generation[-1]
                hidden_state_at_last_token = hidden_state_last_layer[:, -1, :]
                dct['hidden_state_before_gen'] = hidden_state_at_last_token.cpu().float().numpy()
                
                # post-generation hidden states
                hidden_state_post_generation = gends['hidden_states'][-1]
                hidden_state_post_generation = hidden_state_post_generation[-1][0]
                dct['hidden_state_post_gen'] = hidden_state_post_generation.cpu().float().numpy()
        
    np.save(npy_file, dat)
    print(f'Saved npy file {npy_file}')

torch.cuda.empty_cache()